In [38]:
'''
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

'''

'\n!apt-get install -y -qq software-properties-common python-software-properties module-init-tools\n!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null\n!apt-get update -qq 2>&1 > /dev/null\n!apt-get -y install -qq google-drive-ocamlfuse fuse\nfrom google.colab import auth\nauth.authenticate_user()\nfrom oauth2client.client import GoogleCredentials\ncreds = GoogleCredentials.get_application_default()\nimport getpass\n!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL\nvcode = getpass.getpass()\n!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}\n\n'

In [39]:
'''
!mkdir -p drive
!google-drive-ocamlfuse drive
from google.colab import drive
drive.mount('/content/drive')
'''

"\n!mkdir -p drive\n!google-drive-ocamlfuse drive\nfrom google.colab import drive\ndrive.mount('/content/drive')\n"

In [40]:
!pip install torchvision

In [41]:
!nvidia-smi

Thu Jun  3 18:05:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P0    39W / 250W |   2213MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [42]:
#!pip install mxnet-cu112 # 没用着呢
#!pip install minpy

In [43]:
import torch 
x = torch.empty(5,3)
print(x)

tensor([[2.9926e-25, 3.0831e-41, 4.4842e-44],
        [4.4842e-44, 4.4842e-44, 4.4842e-44],
        [4.4842e-44, 4.4842e-44, 4.7644e-44],
        [4.7644e-44, 4.7644e-44, 1.4293e-43],
        [0.0000e+00, 0.0000e+00, 9.1477e-41]])


# main.py 前面import

In [44]:
# -*- coding: UTF-8 -*-
# Python
import os
import random

# Torch
import torch
import numpy as np  #注意这里改了，改成能用GPU的numpy了
import torch.nn as nn
import torch.optim as optim
from numpy.core._multiarray_umath import ndarray
from sklearn.cluster import KMeans
from torch.utils.data import DataLoader
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F
from torch.autograd import Variable
from torch.distributions import Categorical

# Torchvison
import torchvision.transforms as T
import torchvision.models as models
from torchvision.datasets import CIFAR100, CIFAR10
# from influence import *

# Cluster
from sklearn.mixture import GaussianMixture
# Utils
# import visdom
# from tqdm import tqdm

# Custom
#import models.resnet as resnet
#from config import *
#from data.sampler import SubsetSequentialSampler

# import copy

#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"


# config.py

In [45]:
##
# Learning Loss for Active Learning
NUM_TRAIN = 50000 # N
NUM_VAL   = 50000 - NUM_TRAIN
BATCH     = 128  # 128 # B  注意，这里本来是是128
SUBSET    = 25000 # M  # 本来是25000
ADDENDUM  = 2500  # K   # 本来是2500

MARGIN = 1.0  # xi
WEIGHT = 1.0  # 1.0 # lambda

TRIALS = 1
CYCLES = 7   # 本来应该是7

EPOCH = 200   # 本来是200
LR = 0.1    # 0.1 for SGD
MILESTONES = [160]
EPOCHL = 120 # After 120 epochs, stop the gradient from the loss prediction module propagated to the target model

MOMENTUM = 0.9
WDECAY = 5e-4

CLUSTER_NUMS = 10 # 新加的，聚类的cluster 数量
CLUSTER_MAX_ITER = 1000 # 新加的，聚类的迭代次数，本来是10000


''' CIFAR-10 | ResNet-18 | 93.6%
NUM_TRAIN = 50000 # N
NUM_VAL   = 50000 - NUM_TRAIN
BATCH     = 128 # B
SUBSET    = NUM_TRAIN # M
ADDENDUM  = NUM_TRAIN # K

MARGIN = 1.0 # xi
WEIGHT = 0.0 # lambda

TRIALS = 1
CYCLES = 1

EPOCH = 50
LR = 0.1
MILESTONES = [25, 35]
EPOCHL = 40

MOMENTUM = 0.9
WDECAY = 5e-4
'''


' CIFAR-10 | ResNet-18 | 93.6%\nNUM_TRAIN = 50000 # N\nNUM_VAL   = 50000 - NUM_TRAIN\nBATCH     = 128 # B\nSUBSET    = NUM_TRAIN # M\nADDENDUM  = NUM_TRAIN # K\n\nMARGIN = 1.0 # xi\nWEIGHT = 0.0 # lambda\n\nTRIALS = 1\nCYCLES = 1\n\nEPOCH = 50\nLR = 0.1\nMILESTONES = [25, 35]\nEPOCHL = 40\n\nMOMENTUM = 0.9\nWDECAY = 5e-4\n'

# device设置成CPU或GPU 

In [46]:
#
# 加一个device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())

True


# sampler.py

In [47]:
class SubsetSequentialSampler(torch.utils.data.Sampler):
    r"""Samples elements sequentially from a given list of indices, without replacement.

    Arguments:
        indices (sequence): a sequence of indices
    """

    def __init__(self, indices):
        self.indices = indices

    def __iter__(self):
        return (self.indices[i] for i in range(len(self.indices)))

    def __len__(self):
        return len(self.indices)



# main.py 里面的data

In [48]:
##
# Data
train_transform = T.Compose([
    T.RandomHorizontalFlip(),
    T.RandomCrop(size=32, padding=4),
    T.ToTensor(),
    T.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
])

test_transform = T.Compose([
    T.ToTensor(),
    T.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
])

cifar10_train = CIFAR10('../cifar10', train=True, download=True, transform=train_transform)  # specify data path here
cifar10_unlabeled = CIFAR10('../cifar10', train=True, download=True, transform=test_transform)
cifar10_test = CIFAR10('../cifar10', train=False, download=True, transform=test_transform)

##
# Train Utils
iters = 0


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


# resnet.py

In [49]:
#import torch
#import torch.nn as nn
#import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)
        self.linear1 = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out1 = self.layer1(out)
        out2 = self.layer2(out1)
        out3 = self.layer3(out2)
        out4 = self.layer4(out3)
        out5 = F.avg_pool2d(out4, 4)
        out5 = out5.view(out5.size(0), -1)     # [128, 512]
        out = self.linear(out5)
        out_cons = self.linear1(out5)

        return out, out_cons, out5, [out1, out2, out3, out4]


def ResNet18():
    return ResNet(BasicBlock, [2,2,2,2])

def ResNet18_student():
    return ResNet(BasicBlock, [1,1,1,1])

def ResNet34():
    return ResNet(BasicBlock, [3,4,6,3])

def ResNet50():
    return ResNet(Bottleneck, [3,4,6,3])

def ResNet101():
    return ResNet(Bottleneck, [3,4,23,3])

def ResNet152():
    return ResNet(Bottleneck, [3,8,36,3])



# main.py 里面的train_epoch函数

In [50]:
#
def train_epoch(models, criterion, optimizers, dataloaders, epoch, epoch_loss):
    models['backbone'].train()
    global iters

    for data in dataloaders['train']:
        #inputs = data[0].cuda()这是pycharm原来的
        inputs = data[0].to(device)
        #labels = data[1].cuda()这是pycharm原来的
        labels = data[1].to(device)
        iters += 1

        optimizers['backbone'].zero_grad()

        scores, _, _, features_list = models['backbone'](inputs)
        target_loss = criterion(scores, labels)

        m_backbone_loss = torch.sum(target_loss) / target_loss.size(0)
        loss = m_backbone_loss
        loss.backward()
        optimizers['backbone'].step()


# main.py 里面的test函数

In [51]:
#
def test(models, dataloaders, mode='val'):
    assert mode == 'val' or mode == 'test'
    models['backbone'].eval()

    total = 0
    correct = 0
    with torch.no_grad():
        for (inputs, labels) in dataloaders[mode]:
            #inputs = inputs.cuda()这才是prcharm原来的
            inputs = inputs.to(device)
            #labels = labels.cuda()这才是prcharm原来的
            labels = labels.to(device)
            scores, _, _, _ = models['backbone'](inputs)
            _, preds = torch.max(scores.data, 1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()

    return 100 * correct / total

# main.py里面的train函数

In [52]:
#
def train(models, criterion, optimizers, schedulers, dataloaders, num_epochs, epoch_loss, cycle):
    print('>> Train a Model...')
    best_acc = 0.
    checkpoint_dir = os.path.join('./cifar10', 'train', 'weights')
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

    for epoch in range(num_epochs):

        train_epoch(models, criterion, optimizers, dataloaders, epoch, epoch_loss)
        schedulers['backbone'].step()

        # Save a checkpoint
        if epoch % 20 == 0 or epoch == EPOCH - 1:
            acc = test(models, dataloaders, 'test')
            if best_acc < acc:
                best_acc = acc
                torch.save({
                    'epoch': epoch + 1,
                    'state_dict_backbone': models['backbone'].state_dict()
                    # 'state_dict_module': models['module'].state_dict()
                },
                    '%s/active_resnet18_cifar10.pth' % (checkpoint_dir))
            print('Cycle:', cycle + 1, 'Epoch:', epoch + 1, "---", 'Val Acc: {:.3f} \t Best Acc: {:.3f}'.format(acc, best_acc),flush=True)
            
    print('>> Finished.')


# GMM 高斯混合模型

In [53]:
# -*- coding: utf-8 -*-
# ----------------------------------------------------
# Copyright (c) 2017, Wray Zheng. All Rights Reserved.
# Distributed under the BSD License.
# ----------------------------------------------------
'''
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal

DEBUG = True

######################################################
# 调试输出函数
# 由全局变量 DEBUG 控制输出
######################################################
def debug(*args, **kwargs):
    global DEBUG
    if DEBUG:
        print(*args, **kwargs)


######################################################
# 第 k 个模型的高斯分布密度函数
# 每 i 行表示第 i 个样本在各模型中的出现概率
# 返回一维列表
######################################################
def phi(Y, mu_k, cov_k):
    norm = multivariate_normal(mean=mu_k, cov=cov_k)
    return norm.pdf(Y)


######################################################
# E 步：计算每个模型对样本的响应度
# Y 为样本矩阵，每个样本一行，只有一个特征时为列向量
# mu 为均值多维数组，每行表示一个样本各个特征的均值
# cov 为协方差矩阵的数组，alpha 为模型响应度数组
######################################################
def getExpectation(Y, mu, cov, alpha):
    # 样本数
    N = Y.shape[0]
    # 模型数
    K = alpha.shape[0]

    # 为避免使用单个高斯模型或样本，导致返回结果的类型不一致
    # 因此要求样本数和模型个数必须大于1
    assert N > 1, "There must be more than one sample!"
    assert K > 1, "There must be more than one gaussian model!"

    # 响应度矩阵，行对应样本，列对应响应度
    gamma = np.mat(np.zeros((N, K)))

    # 计算各模型中所有样本出现的概率，行对应样本，列对应模型
    prob = np.zeros((N, K))
    for k in range(K):
        prob[:, k] = phi(Y, mu[k], cov[k])
    prob = np.mat(prob)

    # 计算每个模型对每个样本的响应度
    for k in range(K):
        gamma[:, k] = alpha[k] * prob[:, k]
    for i in range(N):
        gamma[i, :] /= np.sum(gamma[i, :])
    return gamma


######################################################
# M 步：迭代模型参数
# Y 为样本矩阵，gamma 为响应度矩阵
######################################################
def maximize(Y, gamma):
    # 样本数和特征数
    N, D = Y.shape
    # 模型数
    K = gamma.shape[1]

    #初始化参数值
    mu = np.zeros((K, D))
    cov = []
    alpha = np.zeros(K)

    # 更新每个模型的参数
    for k in range(K):
        # 第 k 个模型对所有样本的响应度之和
        Nk = np.sum(gamma[:, k])
        # 更新 mu
        # 对每个特征求均值
        mu[k, :] = np.sum(np.multiply(Y, gamma[:, k]), axis=0) / Nk
        # 更新 cov
        cov_k = (Y - mu[k]).T * np.multiply((Y - mu[k]), gamma[:, k]) / Nk
        cov.append(cov_k)
        # 更新 alpha
        alpha[k] = Nk / N
    cov = np.array(cov)
    return mu, cov, alpha


######################################################
# 数据预处理
# 将所有数据都缩放到 0 和 1 之间
######################################################
def scale_data(Y):
    # 对每一维特征分别进行缩放
    for i in range(Y.shape[1]):
        max_ = Y[:, i].max()
        min_ = Y[:, i].min()
        Y[:, i] = (Y[:, i] - min_) / (max_ - min_)
    debug("Data scaled.")
    return Y


######################################################
# 初始化模型参数
# shape 是表示样本规模的二元组，(样本数, 特征数)
# K 表示模型个数
######################################################
def init_params(shape, K):
    N, D = shape
    mu = np.random.rand(K, D)
    cov = np.array([np.eye(D)] * K)
    alpha = np.array([1.0 / K] * K)
    debug("Parameters initialized.")
    debug("mu:", mu, "cov:", cov, "alpha:", alpha, sep="\n")
    return mu, cov, alpha


######################################################
# 高斯混合模型 EM 算法
# 给定样本矩阵 Y，计算模型参数
# K 为模型个数
# times 为迭代次数
######################################################
def GMM_EM(Y, K, times):
    Y = scale_data(Y)
    mu, cov, alpha = init_params(Y.shape, K)
    for i in range(times):
        gamma = getExpectation(Y, mu, cov, alpha)
        mu, cov, alpha = maximize(Y, gamma)
    debug("{sep} Result {sep}".format(sep="-" * 20))
    debug("mu:", mu, "cov:", cov, "alpha:", alpha, sep="\n")
    return mu, cov, alpha
'''

'\nimport numpy as np\nimport matplotlib.pyplot as plt\nfrom scipy.stats import multivariate_normal\n\nDEBUG = True\n\n######################################################\n# 调试输出函数\n# 由全局变量 DEBUG 控制输出\n######################################################\ndef debug(*args, **kwargs):\n    global DEBUG\n    if DEBUG:\n        print(*args, **kwargs)\n\n\n######################################################\n# 第 k 个模型的高斯分布密度函数\n# 每 i 行表示第 i 个样本在各模型中的出现概率\n# 返回一维列表\n######################################################\ndef phi(Y, mu_k, cov_k):\n    norm = multivariate_normal(mean=mu_k, cov=cov_k)\n    return norm.pdf(Y)\n\n\n######################################################\n# E 步：计算每个模型对样本的响应度\n# Y 为样本矩阵，每个样本一行，只有一个特征时为列向量\n# mu 为均值多维数组，每行表示一个样本各个特征的均值\n# cov 为协方差矩阵的数组，alpha 为模型响应度数组\n######################################################\ndef getExpectation(Y, mu, cov, alpha):\n    # 样本数\n    N = Y.shape[0]\n    # 模型数\n    K = alpha.shape[0]\n\n    # 为避免使用单个高斯模型或样本，导致返回结果的类型

# main.py里面的get_uncertainty函数

In [54]:
#
def get_uncertainty(models, unlabeled_loader):
    models['backbone'].eval()
    uncertainty = torch.tensor([]).to(device) #这是原来版本，聚类先不用GPU，用CPU可以用numpy

    with torch.no_grad():
        print(">> Start clustering:")
        for (inputs, labels) in unlabeled_loader:
            inputs = inputs.to(device)
            
            scores, _, total_feature, features = models['backbone'](inputs)
            # total_feature: [SUBSET,512], SUBSET是行，是batch_size，也就是图片个数，512是列，是resnet展平的像素feature个数

            # TODO Use clustering to determine data uncertainty
            input_x = np.array(total_feature.cpu())
            gmm = GaussianMixture(n_components=10).fit(input_x)
            posterior = torch.Tensor(gmm.predict_proba(input_x)).to(device) #后验概率
            uncertainty = Categorical(probs=posterior).entropy()  #计算entropy混乱度，混乱度高说明样本容易被分到很多cluster,是个四不像，
                                                                  # 我觉得四不像的值有时候反而不该选，该选最大的两个概率的差最小的，
                                                                  #或者多个，也就是最容易被分错的
            
            '''
                GMM 算法
            
            '''
            '''
            input_x = scale_data(input_x)
            line_num, column_num = input_x.shape
            result = np.empty(line_num, dtype=np.int)  # line_num个样本的聚类结果
            gamma = np.empty((line_num, CLUSTER_NUMS), dtype=np.float32)  # 我加的
            
            iter = 0
            
            while line_num != 0:  # 迭代聚类计算,这是10000次，这里也可以用while True，质心不变时停止
                iter = iter + 1
            
                mu, cov, alpha = init_params(input_x.shape, CLUSTER_NUMS)
                gamma = getExpectation(input_x, mu, cov, alpha)
                mu, cov, alpha = maximize(input_x, gamma)
                
                index_max = np.argmax(gamma, axis=1)  # 每个样本距离属于的质心索引序号 [line_num, 1]
                if (index_max == result).all() or iter == CLUSTER_MAX_ITER:
                    break;
                #debug("{sep} Result {sep}".format(sep="-" * 20))
                #debug("mu:", mu, "cov:", cov, "alpha:", alpha, sep="\n")


            sorted_gamma = np.sort(gamma)
            two_maxgamma_differ = np.abs(sorted_gamma[:, -1].reshape(line_num, 1)
                                         - sorted_gamma[:, -2].reshape(line_num, 1)) #方法一
            two_maxgamma_differ_torch = torch.from_numpy(two_maxgamma_differ)
            uncertainty = torch.cat((uncertainty, 10.00 / two_maxgamma_differ_torch), 0)
            '''
            
    print(">> Clustering Over:")
    return uncertainty.cpu()


# main.py 里面的main函数

In [55]:

##
# Main
if __name__ == '__main__':

    for trial in range(TRIALS):
        # Initialize a labeled dataset by randomly sampling K=ADDENDUM=1,000 data points from the entire dataset.
        indices = list(range(NUM_TRAIN))
        random.shuffle(indices)
        START = 2 * ADDENDUM
        labeled_set = indices[:START]
        unlabeled_set = indices[START:]

        train_loader = DataLoader(cifar10_train, batch_size=BATCH,  # BATCH
                                  sampler=SubsetRandomSampler(labeled_set),
                                  pin_memory=True)
        test_loader = DataLoader(cifar10_test, batch_size=BATCH)

        dataloaders = {'train': train_loader, 'test': test_loader}

        # Model
        #backbone_net = resnet.ResNet18().cuda() #记住这才是原本pycharm里面的
        backbone_net = ResNet18().to(device)
        
        models = {'backbone': backbone_net}
        torch.backends.cudnn.benchmark = True

        # Active learning cycles
        for cycle in range(CYCLES):
            # Loss, criterion and scheduler (re)initialization
            criterion = nn.CrossEntropyLoss(reduction='none')

            optim_backbone = optim.SGD(models['backbone'].parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WDECAY)

            sched_backbone = lr_scheduler.MultiStepLR(optim_backbone, milestones=MILESTONES)

            optimizers = {'backbone': optim_backbone}
            schedulers = {'backbone': sched_backbone}

            # Training and test
            train(models, criterion, optimizers, schedulers, dataloaders, EPOCH, EPOCHL, cycle)
            acc = test(models, dataloaders, mode='test')
            print('Trial {}/{} || Cycle {}/{} || Label set size {}: Test acc {}'.format(trial + 1, TRIALS, cycle + 1,
                                                                                        CYCLES, len(labeled_set), acc),
                                                                                         flush=True)

            ##
            #  Update the labeled dataset via loss prediction-based uncertainty measurement

            # Randomly sample 10000 unlabeled data points
            random.shuffle(unlabeled_set)
            subset = unlabeled_set[:SUBSET]

            # Create unlabeled dataloader for the unlabeled subset
            unlabeled_loader = DataLoader(cifar10_unlabeled, batch_size=BATCH,
                                          sampler=SubsetSequentialSampler(subset),
                                          # more convenient if we maintain the order of subset
                                          pin_memory=True)

            # Measure uncertainty of each data points in the subset
            uncertainty = get_uncertainty(models, unlabeled_loader)
            #print("main uncertainty: ", uncertainty)
            uncertainty = uncertainty.T
            # Index in ascending order
            arg = np.argsort(uncertainty).numpy().tolist()
            #print("main arg: ", arg) 
            
            # Update the labeled dataset and the unlabeled dataset, respectively
            labeled_set += list(torch.tensor(subset)[arg][-ADDENDUM:].numpy())  # select largest loss
            unlabeled_set = list(torch.tensor(subset)[arg][:-ADDENDUM].numpy()) + unlabeled_set[SUBSET:]

            # labeled_set += list(torch.tensor(subset)[arg][:ADDENDUM].numpy())  # select smallest influence
            # unlabeled_set = list(torch.tensor(subset)[arg][ADDENDUM:].numpy()) + unlabeled_set[SUBSET:]

            # Create a new dataloader for the updated labeled dataset
            dataloaders['train'] = DataLoader(cifar10_train, batch_size=BATCH,  # BATCH
                                              sampler=SubsetRandomSampler(labeled_set),
                                              pin_memory=True)

        # Save a checkpoint
        torch.save({
            'trial': trial + 1,
            'state_dict_backbone': models['backbone'].state_dict()
            # 'state_dict_module': models['module'].state_dict()
        },
            './cifar10/train/weights/active_resnet18_cifar10_trial{}.pth'.format(trial))

        print('---------------------------Current Trial is done-----------------------------',flush=True)


>> Train a Model...
Cycle: 1 Epoch: 1 --- Val Acc: 10.990 	 Best Acc: 10.990
Cycle: 1 Epoch: 21 --- Val Acc: 44.060 	 Best Acc: 44.060
Cycle: 1 Epoch: 41 --- Val Acc: 55.710 	 Best Acc: 55.710
Cycle: 1 Epoch: 61 --- Val Acc: 63.450 	 Best Acc: 63.450
Cycle: 1 Epoch: 81 --- Val Acc: 68.310 	 Best Acc: 68.310
Cycle: 1 Epoch: 101 --- Val Acc: 69.060 	 Best Acc: 69.060
Cycle: 1 Epoch: 121 --- Val Acc: 63.240 	 Best Acc: 69.060
Cycle: 1 Epoch: 141 --- Val Acc: 69.940 	 Best Acc: 69.940
Cycle: 1 Epoch: 161 --- Val Acc: 77.710 	 Best Acc: 77.710
Cycle: 1 Epoch: 181 --- Val Acc: 79.220 	 Best Acc: 79.220
Cycle: 1 Epoch: 200 --- Val Acc: 78.810 	 Best Acc: 79.220
>> Finished.
Trial 1/1 || Cycle 1/7 || Label set size 5000: Test acc 78.81
>> Start clustering:
>> Clustering Over:
>> Train a Model...
Cycle: 2 Epoch: 1 --- Val Acc: 75.810 	 Best Acc: 75.810
Cycle: 2 Epoch: 21 --- Val Acc: 72.140 	 Best Acc: 75.810
Cycle: 2 Epoch: 41 --- Val Acc: 73.810 	 Best Acc: 75.810
Cycle: 2 Epoch: 61 --- Val A